# Justice scraper 1.0

## created by Adam Tůma

Justice scraper can scrape information of companies once provided with a list of IČOs

The script will return dataset obtaining the following information, with index values as the IČO of given company:
- Company name / Název společnosti
- Date of registration / Datum vzniku
- File number / Spisová značka
- Address / Sídlo
- Legal form / Právní forma


In [244]:
import scrapy
import requests
import pandas as pd
import geopy
import time
import folium
from bs4 import BeautifulSoup
from scrapy import Selector
from geopy.geocoders import Nominatim
from scrapy.crawler import CrawlerProcess

In [241]:
text_icos = open('top100_icos.txt', 'r')
icos = [line.strip() for line in text_icos]
text_icos.close()

print(icos)

['00177041', '28356250', '45274649', '26185610', '25938002', '61672190', '27773035', '28477090', '63474808', '45788235', '60193531', '43872247', '49450301', '00014915', '70994226', '63080737', '26513528', '18050646', '64945880', '26463318', '49903209', '60193336', '27082440', '26450691', '49241214', '64949681', '29259428', '00268577', '45359326', '25654012', '25702556', '61672599', '48171131', '63473291', '00000931', '46995129', '26919389', '27386732', '27636801', '46678735', '15890554', '60193913', '27214265', '02176475', '00005886', '04084063', '14707420', '03592880', '01759299', '25663135', '48365289', '46711953', '05735025', '45357366', '25029673', '14803534', '47114983', '48038687', '29060770', '14706725', '14803534', '25077830', '49240030', '60193492', '27295567', '28987322', '60838744', '00534111', '26271303', '41189671', '14888742', '26128209', '26455137', '25133152', '28196678', '45193410', '28234642', '45274924', '26440181', '64833054', '28244532', '43005012', '26161516', '45

In [ ]:
ico = ["27201121","27201121"]
start_urls = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"

html = requests.get(url).content

html

In [136]:
def get_url(ico):
    url_0 = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
    html_subjektID = requests.get(url_0).content
    sel_id = Selector(text=html_subjektID)
    vypis = sel_id.xpath("//li/a[contains(text(),'Výpis platných')]/@href").extract()
    subjektID = vypis[0].split('subjektId=')[1].split('&')[0]
    url = f'https://or.justice.cz/ias/ui/rejstrik-firma.vysledky?subjektId={subjektID}&typ=PLATNY'
    return url

urls = []

def get_urls(icos):
    for i in icos:
        urls.append(get_url(i))

def get_soup(ico):
    url = get_url(ico)
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    return soup




In [ ]:
ico = "27082440"
url = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
url2 = "https://or.justice.cz/ias/ui/rejstrik-firma.vysledky?subjektId=701502&typ=PLATNY"
html = requests.get(url2).content
soup = BeautifulSoup(html, 'lxml')

match = soup.find(text="Obchodní firma: ").findNext('span')
match2 = match.findNext('span')


match3 = soup.find(text="Identifikační číslo: ").findNext('span').text

match4 = soup.findAll(class_="nounderline")
match4[5].findNext('span')
print(soup.prettify())



## The function getBasicInfo obtains basic 5 datapoints that are the same across all companies on justice.cz

In [246]:
def getBasicInfo(icos):
    df = pd.DataFrame(index=icos, columns = ['Název společnosti', 'Datum vzniku', 'Spisová značka', 'Sídlo', 'Právní forma'])
    for ico in icos:
        soup = get_soup(ico)

        date = soup.find(text="Datum vzniku a zápisu:").findNext('div').findNext('div').text
        df.loc[ico,'Datum vzniku'] = date

        spis = soup.find(text="Spisová značka: ").findNext('span').text
        df.loc[ico,'Spisová značka'] = spis

        nazev = soup.find(class_="nounderline").findNext(class_="nounderline").findNext('span').text    #here we take different approach as the object is sometimes Obchodní firma and sometimes different
        df.loc[ico,'Název společnosti'] = nazev

        sidlo = soup.find(text="Sídlo: ").findNext('span').findNext('span').text
        df.loc[ico,'Sídlo'] = sidlo

        pravni_forma = soup.find(text="Právní forma: ").findNext('span').text
        df.loc[ico,'Právní forma'] = pravni_forma
        time.sleep(1)
    return df

#df = getBasicInfo(icos)
df

,Název společnosti,Datum vzniku,Spisová značka,Sídlo,Právní forma
00177041,ŠKODA AUTO a.s.,20. listopadu 1990,B 332 vedená u Městského soudu v Praze,"tř. Václava Klementa 869, Mladá Boleslav II, 2...",Akciová společnost
28356250,"Energetický a průmyslový holding, a.s.",10. srpna 2009,B 21747 vedená u Městského soudu v Praze,"Pařížská 130/26, Josefov, 110 00 Praha 1",Akciová společnost
45274649,"ČEZ, a. s.",6. května 1992,B 1581 vedená u Městského soudu v Praze,"Praha 4, Duhová 2/1444, PSČ 14053",Akciová společnost
26185610,"AGROFERT, a.s.",1. července 2000,B 6626 vedená u Městského soudu v Praze,"Pyšelská 2327/2, Chodov, 149 00 Praha 4",Akciová společnost
25938002,FOXCONN CZ s.r.o.,18. května 2000,C 16095 vedená u Krajského soudu v Hradci Králové,"U Zámečku 27, Pardubičky, 530 03 Pardubice",Společnost s ručením omezeným
...,...,...,...,...,...
63145936,Benteler ČR s.r.o.,12. července 1995,C 22895 vedená u Krajského soudu v Ústí nad Labem,"Školní 713, 463 31 Chrastava",Společnost s ručením omezeným
49284975,"KIEKERT-CS, s.r.o.",1. června 1993,C 4473 vedená u Krajského soudu v Hradci Králové,"Jaselská 593, 535 01 Přelouč",Společnost s ručením omezeným
02397811,Adient Bor s.r.o.,3. prosince 2013,C 36301 vedená u Krajského soudu v Plzni,"Nová Hospoda 29, 348 02 Bor",Společnost s ručením omezeným
26307391,Miele technika s.r.o.,13. prosince 2002,C 28029 vedená u Krajského soudu v Ostravě,"Uničov, Šumperská 1348, PSČ 78391",Společnost s ručením omezeným


In [247]:
def getCoordinates(df):
    ### FUNCTION TO OBTAIN GPS COORDINATES FROM ADDRES ###
    locator = Nominatim(user_agent='justice_scraper')
    list_address = df.loc[:,'Sídlo'].tolist()
    latitudes = []
    longitudes = []
    for i in list_address:
        split = i.split(',')                        # first we try to obtain coordinates from the first 2 splits in the addres, usually the street name and number + city
        address = split[0]+', '+split[1]
        location = locator.geocode(address)
        if not location:                            # sometimes the address is wierdly formatted and the second info is the postal code, which results in not finding any coordinates, we then try only from the first split
            split1 = i.split(',')
            address1 = split1[0]
            lat1 = locator.geocode(address1).latitude
            lon1 = locator.geocode(address1).longitude
            latitudes.append(lat1)
            longitudes.append(lon1)
        else: 
            lat = locator.geocode(address).latitude
            lon = locator.geocode(address).longitude
            latitudes.append(lat)
            longitudes.append(lon)
    df['latitude'] = latitudes
    df['longitude'] = longitudes
    return df

df = getCoordinates(df)
df

,Název společnosti,Datum vzniku,Spisová značka,Sídlo,Právní forma,latitude,longitude
00177041,ŠKODA AUTO a.s.,20. listopadu 1990,B 332 vedená u Městského soudu v Praze,"tř. Václava Klementa 869, Mladá Boleslav II, 2...",Akciová společnost,50.426931,14.918648
28356250,"Energetický a průmyslový holding, a.s.",10. srpna 2009,B 21747 vedená u Městského soudu v Praze,"Pařížská 130/26, Josefov, 110 00 Praha 1",Akciová společnost,50.090427,14.418938
45274649,"ČEZ, a. s.",6. května 1992,B 1581 vedená u Městského soudu v Praze,"Praha 4, Duhová 2/1444, PSČ 14053",Akciová společnost,49.940474,14.423769
26185610,"AGROFERT, a.s.",1. července 2000,B 6626 vedená u Městského soudu v Praze,"Pyšelská 2327/2, Chodov, 149 00 Praha 4",Akciová společnost,50.038638,14.497542
25938002,FOXCONN CZ s.r.o.,18. května 2000,C 16095 vedená u Krajského soudu v Hradci Králové,"U Zámečku 27, Pardubičky, 530 03 Pardubice",Společnost s ručením omezeným,50.029444,15.803469
...,...,...,...,...,...,...,...
63145936,Benteler ČR s.r.o.,12. července 1995,C 22895 vedená u Krajského soudu v Ústí nad Labem,"Školní 713, 463 31 Chrastava",Společnost s ručením omezeným,50.821435,14.970132
49284975,"KIEKERT-CS, s.r.o.",1. června 1993,C 4473 vedená u Krajského soudu v Hradci Králové,"Jaselská 593, 535 01 Přelouč",Společnost s ručením omezeným,50.037363,15.580635
02397811,Adient Bor s.r.o.,3. prosince 2013,C 36301 vedená u Krajského soudu v Plzni,"Nová Hospoda 29, 348 02 Bor",Společnost s ručením omezeným,49.746047,12.764198
26307391,Miele technika s.r.o.,13. prosince 2002,C 28029 vedená u Krajského soudu v Ostravě,"Uničov, Šumperská 1348, PSČ 78391",Společnost s ručením omezeným,49.782249,17.120397


In [251]:
map1 = folium.Map(
    location=[49.861464, 15.496766],
    tiles='cartodbpositron',
    zoom_start=8,
)

df.apply(lambda row:folium.Marker(location=[row["latitude"], row["longitude"]], popup=row['Název společnosti']).add_to(map1), axis=1)

map1.save(outfile= "map.html")

filtering the objects so they only include non-empty strings

In [ ]:
filter_list=[]

for article in soup.findAll(class_="nounderline"):
    article_text = article.text
    if not article_text:
        pass
    else: 
        filter_list.append(article)

filter_list


In [43]:
prvni = filter_list[16].findNext('span')
prvni_text = prvni.text
druhy = prvni.findNext('span')
druhy_text = druhy.text
treti = druhy.findNext('span')
treti_text = treti.text
prvni_text

''

In [86]:
testik = soup.find(class_="nounderline").findNext(class_="nounderline").findNext('span').text
print(testik)


Alza.cz a.s.


In [ ]:
info = []

def info_attacher(list):
    for object in list:
        while True:
            if object.text == (object+1).text:
                pass
            elif not i.text:
                pass
            else:
                info.append(i.text)

info_attacher(filter_list)
            

In [199]:
locator = Nominatim(user_agent='justice_scraper')
location = locator.geocode('Praha 4 Duhová 2/1444')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))



Latitude = 49.9404735, Longitude = 14.4237687
